In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [4]:
def squad_json_to_dataframe_dev(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
#     ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
#     js['q_idx'] = ndx
    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [5]:
input_file_path = '/content/drive/MyDrive/dev.txt'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_dev(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


shape of the dataframe is (10570, 5)
Done


In [24]:
dev.head()

,id,question,context,answers,c_id
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'Carolina Panth...",0
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'Santa Clara, C...",0
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold'}, {'answ...",0


In [6]:
with open('/content/drive/MyDrive/eval_predictions.json', mode='rt', encoding='utf-8-sig') as f:
    dev_pred = json.load(f)

In [44]:
dev_pred_df=pd.DataFrame(zip(list(dev_pred.keys()), list(dev_pred.values())), columns=['id', 'answer'])
merge = pd.merge(dev_pred_df,dev, on='id')
merge=merge[['id', 'answer', 'answers']]
merge

,id,answer,answers
0,56be4db0acb8001400a502ec,National Football League,"[{'answer_start': 177, 'text': 'Denver Broncos..."
1,56be4db0acb8001400a502ed,Carolina Panthers,"[{'answer_start': 249, 'text': 'Carolina Panth..."
2,56be4db0acb8001400a502ee,Levi's Stadium,"[{'answer_start': 403, 'text': 'Santa Clara, C..."
3,56be4db0acb8001400a502ef,National Football League,"[{'answer_start': 177, 'text': 'Denver Broncos..."
4,56be4db0acb8001400a502f0,golden,"[{'answer_start': 488, 'text': 'gold'}, {'answ..."
...,...,...,...
10565,5737aafd1c456719005744fb,pound-force,"[{'answer_start': 82, 'text': 'kilogram-force'..."
10566,5737aafd1c456719005744fc,"kilopond), is the force exerted by standard gr...","[{'answer_start': 114, 'text': 'kilopond'}, {'..."
10567,5737aafd1c456719005744fd,1 m·s−2,"[{'answer_start': 274, 'text': 'slug'}, {'answ..."
10568,5737aafd1c456719005744fe,kip,"[{'answer_start': 712, 'text': 'kip'}, {'answe..."


In [ ]:
EM=0
f1_sum=0

cnt=0
correct=[]
uncorrect=[]
f = open("./log.txt",'a')

for i in range(0, merge.shape[0]):

  answer=merge.loc[i]['answer'].lower().split(" ")
  goldens=[]
  for j in range(len(merge.loc[i]['answers'])):
    goldens.append(merge.loc[i]['answers'][j]['text'].lower().split(" "))

  print("Answer: ", answer)
  f.write("Answer: "+str(answer)+"\n")
  print("Golden: ", goldens)
  f.write("Golden: "+str(goldens)+"\n")

  if answer in goldens:
    EM+=1
    print("correct")
    f.write("correct"+"\n")
    correct.append((answer, goldens))
  else:
    print("uncorrect")
    f.write("uncorrect"+"\n")
    uncorrect.append((answer,goldens))
  f1=0
  for i in range(len(goldens)):
    precision=(len(answer)- len(set(answer)-set(goldens[i]))) / len(answer)
    recall=(len(goldens[i])- len(set(goldens[i])-set(answer))) / len(goldens[i])
    
    if precision+recall!=0:
      f1=max(f1, (precision * recall * 2) / (precision + recall))
    else:
      f1=0
  f1_sum+=f1
  print("precision: "+str(precision)+"\n")
  print("recall: "+str(recall)+"\n")
  print("f1 score: "+str(f1)+"\n")

  f.write("precision: "+str(precision)+"\n")
  f.write("recall: "+str(recall)+"\n")
  f.write("f1 score: "+str(f1)+"\n")

  print("")
  f.write("\n")
  cnt+=1
  if cnt%100==0:
    print(cnt)
f.close()
print("EM:", EM)
print("f1 sum:", f1_sum)

In [53]:
EM/dev.shape[0]

0.6943235572374645

In [54]:
f1_sum/dev.shape[0]

0.7742255897892306

In [5]:
input_file_path = '/content/drive/MyDrive/dev-v2.0.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_dev(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


shape of the dataframe is (11873, 5)
Done


In [6]:
dev.head()

,id,question,context,answers,c_id
0,56ddde6b9a695914005b9628,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': 'France', 'answer_start': 159}, {'te...",0
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': '10th and 11th centuries', 'answer_s...",0
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': 'Denmark, Iceland and Norway', 'answ...",0
3,56ddde6b9a695914005b962b,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': 'Rollo', 'answer_start': 308}, {'tex...",0
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,"[{'text': '10th century', 'answer_start': 671}...",0


In [7]:
with open('/content/drive/MyDrive/eval_predictions_v2.json', mode='rt', encoding='utf-8-sig') as f:
    dev_pred = json.load(f)

In [8]:
dev_pred_df=pd.DataFrame(zip(list(dev_pred.keys()), list(dev_pred.values())), columns=['id', 'answer'])
merge = pd.merge(dev_pred_df,dev, on='id')
merge=merge[['id', 'answer', 'answers']]
merge

,id,answer,answers
0,56ddde6b9a695914005b9628,France,"[{'text': 'France', 'answer_start': 159}, {'te..."
1,56ddde6b9a695914005b9629,10th and 11th centuries,"[{'text': '10th and 11th centuries', 'answer_s..."
2,56ddde6b9a695914005b962a,"Denmark, Iceland and Norway","[{'text': 'Denmark, Iceland and Norway', 'answ..."
3,56ddde6b9a695914005b962b,Rollo,"[{'text': 'Rollo', 'answer_start': 308}, {'tex..."
4,56ddde6b9a695914005b962c,10th,"[{'text': '10th century', 'answer_start': 671}..."
...,...,...,...
11868,5737aafd1c456719005744ff,kilogram-force,"[{'text': 'sthène', 'answer_start': 665}, {'te..."
11869,5ad28ad0d7d075001a4299cc,pound-force,[]
11870,5ad28ad0d7d075001a4299cd,kilogram-force,[]
11871,5ad28ad0d7d075001a4299ce,kilogram-force,[]


In [18]:
blank=0
for i in range(merge.shape[0]):
  if merge['answers'][i]==[]:
    blank+=1
    print(merge['answers'][i])
print(blank)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [22]:
EM=0
f1_sum=0

cnt=0
correct=[]
uncorrect=[]
f = open("./log.txt",'a')

for i in range(0, merge.shape[0]):
  if merge.loc[i]['answers']==[]:
    continue
  answer=merge.loc[i]['answer'].lower().split(" ")
  goldens=[]
  for j in range(len(merge.loc[i]['answers'])):
    goldens.append(merge.loc[i]['answers'][j]['text'].lower().split(" "))

  print("Answer: ", answer)
  f.write("Answer: "+str(answer)+"\n")
  print("Golden: ", goldens)
  f.write("Golden: "+str(goldens)+"\n")

  if answer in goldens:
    EM+=1
    print("correct")
    f.write("correct"+"\n")
    correct.append((answer, goldens))
  else:
    print("uncorrect")
    f.write("uncorrect"+"\n")
    uncorrect.append((answer,goldens))
  f1=0
  for i in range(len(goldens)):
    precision=(len(answer)- len(set(answer)-set(goldens[i]))) / len(answer)
    recall=(len(goldens[i])- len(set(goldens[i])-set(answer))) / len(goldens[i])
    
    if precision+recall!=0:
      f1=max(f1, (precision * recall * 2) / (precision + recall))
    else:
      f1=0
  f1_sum+=f1
  print("precision: "+str(precision)+"\n")
  print("recall: "+str(recall)+"\n")
  print("f1 score: "+str(f1)+"\n")

  f.write("precision: "+str(precision)+"\n")
  f.write("recall: "+str(recall)+"\n")
  f.write("f1 score: "+str(f1)+"\n")

  print("")
  f.write("\n")
  cnt+=1
  if cnt%100==0:
    print(cnt)
f.close()
print("EM:", EM/cnt)
print("f1 sum:", f1_sum/cnt)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
f1 score: 1.0


Answer:  ['masovian', 'primeval', 'forest']
Golden:  [['masovian', 'primeval', 'forest'], ['masovian', 'primeval', 'forest'], ['masovian', 'primeval', 'forest']]
correct
precision: 1.0

recall: 1.0

f1 score: 1.0


Answer:  ['kabaty', 'forest']
Golden:  [['kabaty'], ['kabaty', 'forest'], ['kabaty']]
correct
precision: 0.5

recall: 1.0

f1 score: 1.0


Answer:  ['two']
Golden:  [['two'], ['two'], ['two']]
correct
precision: 1.0

recall: 1.0

f1 score: 1.0


Answer:  ['13']
Golden:  [['13'], ['13'], ['13']]
correct
precision: 1.0

recall: 1.0

f1 score: 1.0


Answer:  ['15', 'kilometres']
Golden:  [['15', 'kilometres'], ['15', 'kilometres'], ['about', '15', 'kilometres']]
correct
precision: 1.0

recall: 0.6666666666666666

f1 score: 1.0


Answer:  ['otter,', 'beaver', 'and', 'hundreds', 'of', 'bird', 'species']
Golden:  [['otter,', 'beaver', 'and', 'hundreds', 'of', 'bird', 'species'], ['otter,', 'beaver', 'and', 'hundreds', 'of', 'bird